# Pipeline for BIAPT lab EEG Preprocessing: 
#### inspired by: https://github.com/hoechenberger/pybrain_mne/
#### adapted by: Charlotte Maschke
#### This pipeline uses MNE Python to preprocess EEG data: Plese go here: 
####                                https://mne.tools/stable/overview/index.html
####  for more documentation on MNE Python

## Some setup and import

In [1]:
import matplotlib
import mne_bids
import pathlib
import mne

# Ensure Matplotlib uses the Qt5Agg backend, 
# which is the best choice for MNE-Python's 
# interactive plotting functions.
matplotlib.use('Qt5Agg')

import matplotlib.pyplot as plt

### Which subject do you want to preprocess? 

In [2]:
ID = "xxYY"
task = "Rest"
BIDS_dir = "../BIDS_testdata/"

In [3]:
raw_path = "{}/source/sub-{}/eeg/sub-{}_task-{}.mff".format(BIDS_dir,ID,ID,task)
raw_path

'../BIDS_testdata//source/sub-xxYY/eeg/sub-xxYY_task-Rest.mff'

## Load the raw data!

In [4]:
raw = mne.io.read_raw_egi(raw_path)
raw

Reading EGI MFF Header from C:\Users\User\Documents\GitHub\EEG_Preprocessing\BIDS_testdata\source\sub-xxYY\eeg\sub-xxYY_task-Rest.mff...
    Reading events ...
    Assembling measurement info ...


<RawMff | signal1.bin, 129 x 160543 (642.2 s), ~178 kB, data not loaded>

## Crop the data

crop on 2 ends:  .crop(tmin = 000 , tmax = 000)  
crop beginning:  .crop(tmin = 100 )  
crop end only :  .crop(tmax = 100)

#### the values here are in seconds ! 

In [5]:
# show the duration of your signal
print("Length of Signal in min: ")
raw.times[-1]/60

Length of Signal in min: 


10.7028

In [6]:
# this can be used to transform minutes to milliseconds
ms_min = 1  * 60 
ms_max = 3  * 60

In [63]:
# input here your minimal and maximal time you want to keep 
raw_cropped = raw.copy().crop(tmin = ms_min)
#raw_cropped = raw.copy().crop(tmin = ms_min, tmax = ms_max)

### Keep only the EEG

In [64]:
# this is to load EEG. If you want to load other stuff please refer to the website documetation
eeg_cropped = raw_cropped.pick_types(eeg = True)
print('Number of channels in EEG:')
len(eeg_cropped.ch_names)

Number of channels in EEG:


129

## Filter the data

In [65]:
# load actual data into system (before it was only metadata)
eeg_cropped.load_data()

# filter the data between 0.1 to 55 Hz
eeg_cropped_filtered = eeg_cropped.filter(l_freq=0.1, h_freq = 55)

# notch filter the data between 0.1 to 55 Hz
eeg_filtered = eeg_cropped_filtered.copy().notch_filter(freqs=60)

Reading 0 ... 145542  =      0.000 ...   582.168 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 55 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 55.00 Hz
- Upper transition bandwidth: 13.75 Hz (-6 dB cutoff frequency: 61.88 Hz)
- Filter length: 8251 samples (33.004 sec)

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
-

## Mark channels as bad

In [103]:
eeg_filtered.plot()


<MNEBrowseFigure size 1920x1046 with 4 Axes>

Channels marked as bad: ['E56', 'E126', 'E44', 'E48', 'E49', 'E43']


In [11]:
STOP

NameError: name 'STOP' is not defined

Mark an additional EEG channel as bad and view the topoplot.

In [98]:
eeg_filtered.info['bads']

['E56', 'E126', 'E44', 'E48', 'E49', 'E43']

### Average Reference the data

In [99]:
# use the average of all channels as reference
eeg_avg_ref = eeg_filtered.copy().set_eeg_reference(ref_channels='average')
#eeg_avg_ref.plot()

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.


### Remove Non-Brain Electrodes

In [100]:
non_brain_el = ['E127', 'E126', 'E17', 'E21', 'E14', 'E25', 'E8', 'E128', 'E125', 'E43', 'E120', 'E48', 
                'E119', 'E49', 'E113', 'E81', 'E73', 'E88', 'E68', 'E94', 'E63', 'E99', 'E56', 'E107' ]

#only add non-brain channels if not already part of noisy channels
for e in non_brain_el: 
    if e not in eeg_avg_ref.info['bads']:
        eeg_avg_ref.info['bads'].append(e)

#eeg_avg_ref.info['bads']
eeg_brainonly= eeg_avg_ref.copy()

## Let's visualize the PSD

In [102]:
fig = eeg_brainonly.plot_psd(fmax = 60)
fig.savefig('{}_{}_PSD.jpg'.format(ID,task))

Effective window size : 8.192 (s)


### Save image with removed channels

In [71]:
eeg_brainonly.plot_sensors(ch_type='eeg')
plt.savefig('{}_{}_badchannels.jpg'.format(ID,task))
plt.close()


### Save the data in BIDS format

In [60]:
import os
out_dir = pathlib.Path(BIDS_dir,"sub-{}/eeg/".format(ID))

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
out_path = pathlib.Path(out_dir, "sub-{}_task-{}.fif".format(ID,task))

In [61]:
eeg_brainonly.save(out_path,overwrite=True)

Overwriting existing file.
Writing C:\Users\User\Documents\GitHub\EEG_Preprocessing\BIDS_testdata\sub-xxYY\eeg\sub-xxYY_task-Rest.fif


<ipython-input-61-f9268ae67915>:1: RuntimeWarning: This filename (C:\Users\User\Documents\GitHub\EEG_Preprocessing\BIDS_testdata\sub-xxYY\eeg\sub-xxYY_task-Rest.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_brainonly.save(out_path,overwrite=True)


Closing C:\Users\User\Documents\GitHub\EEG_Preprocessing\BIDS_testdata\sub-xxYY\eeg\sub-xxYY_task-Rest.fif
[done]


# Epoch the data and remove noisy epochs

In [95]:
epochs = mne.make_fixed_length_epochs(eeg_brainonly, duration = 10,overlap=0, preload=False)

Not setting metadata
Not setting metadata
58 matching events found
No baseline correction applied
0 projection items activated


In [96]:
epochs.plot()

Loading data for 58 events and 2500 original time points ...
0 bad epochs dropped
Loading data for 38 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...


<MNEBrowseFigure size 1920x1046 with 4 Axes>

Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 events and 2500 original time points ...
Loading data for 20 even

In [ ]:
STOP

In [74]:
epochs.save(pathlib.Path('out_data') / 'epochs_{}_{}.fif'.format(ID,task), 
            overwrite=True)

Overwriting existing file.
Loading data for 1 events and 2500 original time points ...
Loading data for 52 events and 2500 original time points ...


<ipython-input-74-3f281627ff0e>:1: RuntimeWarning: This filename (out_data\epochs_xxYY_Rest.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(pathlib.Path('out_data') / 'epochs_{}_{}.fif'.format(ID,task),
